<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">Colab KSO Tutorial #1: Check and update csv files</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Jun 27th, 2022</h5>

# Set up and requirements

## Install kso_data_management and its requirements

In [ ]:
# Clone koster_data_management repo
!git clone --recurse-submodules -b dev https://github.com/ocean-data-factory-sweden/koster_data_management.git
!pip install -r koster_data_management/requirements.txt

# Install ipysheet version compatible in colab
!pip install ipysheet==0.4.4

# Restart the session to load the latest packages
exit()

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
os.chdir("koster_data_management/tutorials")
sys.path.append('..')

# Enable third-party widgets(ipysheet)
from google.colab import output
output.enable_custom_widget_manager()

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t1_utils as t1

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

## Initiate database

In [ ]:
project = p_utils.find_project(project_name=project_name.value)
db_info_dict = t_utils.initiate_db(project)

# Review Sites

## Map sites and metadata

In [ ]:
kso_map = t1.map_site(db_info_dict, project)
kso_map

## Update sites metadata

You can update the contents of the cells in the spreadsheet below

In [ ]:
sites_sheet = t1.open_sites_csv(db_info_dict)
sites_sheet

### Confirm the changes

In [ ]:
# Display the changes
highlight_changes, sites_sheet_df = t1.display_changes(db_info_dict,
                   isheet = sites_sheet, 
                   local_csv = "local_sites_csv"
                   )
highlight_changes

In [ ]:
# Confirm or deny the changes
t1.update_csv(db_info_dict, 
              project,
              sheet_df = sites_sheet_df,
              local_csv = "local_sites_csv",
              serv_csv = "server_sites_csv"
              )

# Review Movies

In [ ]:
WORK IN PROGRESS AFTER THIS

In [ ]:
movies_df_sheet = t1.open_movies_csv(db_initial_info)
movies_df_sheet

In [ ]:
movies_df = t1.check_movies_csv(db_info_dict, movies_df_sheet, project)

In [ ]:
#END